In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from scipy import stats
from sklearn.model_selection import train_test_split
import warnings
import matplotlib.pyplot as plt
import datetime

In [3]:
pd.set_option('display.max_rows', 99999)
pd.set_option('display.max_columns', 500)
warnings.filterwarnings("ignore")

In [4]:
epc_49 = pd.read_csv('2022_49.csv')
##2022_49.csv contains only chosen features(house characteristics) from the original dataset
epc_49.shape

(54495, 49)

In [5]:
epc_49.head(1)

Total floor area (m²)  Total current energy costs over 3 years (£)  \
0                     45                                         3771   

   Current energy efficiency rating Current energy efficiency rating band  \
0                                46                                     E   

   Current Environmental Impact Rating  \
0                                   43   

  Current Environmental Impact Rating Band  \
0                                        E   

                                        Improvements  \
0  Description: High heat retention storage heate...   

                                    WALL_DESCRIPTION  \
0  Sandstone or limestone, as built, no insulatio...   

                   ROOF_DESCRIPTION          FLOOR_DESCRIPTION  \
0  Pitched, 150 mm loft insulation   (another dwelling below)    

                 WINDOWS_DESCRIPTION     MAINHEAT_DESCRIPTION  \
0  Description: Fully double glazed   Room heaters, electric    

                MAINHEATCONT_DESCRIPTION SECONDHEAT_DESCRIPTION  \
0  Programmer and appliance thermostats                   None    

            HOTWATER_DESCRIPTION                       LIGHTING_DESCRIPTION  \
0  Electric immersion, off-peak   Low energy lighting in all fixed outlets    

  AIR_TIGHTNESS_DESCRIPTION  Current heating costs over 3 years (£)  \
0                       NaN                                    2478   

   Current hot water costs over 3 years (£)  \
0                                      1176   

   Current lighting costs over 3 years (£)  Space Heating  Water Heating  \
0                                      117           5270           3058   

  Part 1 Construction Age Band  Part 1 Floor 0 Room Height  \
0                  before 1919                         3.1   

   Fixed Lighting Outlets Count  Low Energy Lighting Outlets Count  \
0                             6                                  6   

   Low Energy Lighting % Flat Level Flat Location  Glazed Area  \
0                    100  top floor             2            1   

   Habitable Room Count Heat Loss Corridor  Heated Room Count  \
0                     2  unheated corridor                  2   

  Main Heating 1 Category     Main Heating 1 Fuel Type  \
0            room heaters  electricity (not community)   

   Main Heating 1 Control Mechanical Ventilation Meter Type  \
0                    2603                natural       dual   

   Multiple Glazed Proportion                 Multiple Glazing Type  \
0                         100  double glazing, unknown install date   

   Open Fireplaces Count                                Photovoltaic Supply  \
0                      0  Array: Roof Area: 0%; Connection: not applicab...   

  Solar Water Heating          Tenure   Transaction Type  \
0                   N  owner-occupied  none of the above   

   Unheated Corridor Length  Wind Turbines Count   Built Form Property Type  
0                       4.5                    0  Mid-Terrace          Flat

In [6]:
##drop Photovoltaic Supply and improvements ( because they are correlated(reduce) with energy consumption) 
epc_49 = epc_49.drop(['Photovoltaic Supply', 'Improvements'], axis=1)

In [7]:
#drop columns having more than 20% null
epc_49 = epc_49.replace([0,' ','NULL'],np.nan)
null_percentage = epc_49.isnull().sum()/epc_49.shape[0]*100

col_to_drop = null_percentage[null_percentage>20].keys()
epc_49 = epc_49.drop(col_to_drop, axis=1)

In [8]:
#remove outliers using stats
epc_num = epc_49.select_dtypes(include= ['number'])
min = 0
maxPercentile = 0.9999
for _ in epc_num:
    epc_49[_] =  epc_49[_].clip(0,(epc_49[_].quantile(maxPercentile)))

In [9]:
#impute nulls with most frequent values
epc_49 = epc_49.apply(lambda x: x.fillna(x.value_counts().index[0]))

In [10]:
epc_49 =epc_49.drop(['Current Environmental Impact Rating','Current Environmental Impact Rating Band', 
                    'Space Heating','Water Heating', 'Main Heating 1 Category', 'Main Heating 1 Fuel Type',
                     'Main Heating 1 Control','Multiple Glazing Type', 'Multiple Glazed Proportion', 
                    'Current energy efficiency rating'
                    ], axis=1)

Descriptive features handling

In [11]:
descriptive_columns = ['WALL_DESCRIPTION','ROOF_DESCRIPTION','FLOOR_DESCRIPTION','WINDOWS_DESCRIPTION',
                       'MAINHEAT_DESCRIPTION','MAINHEATCONT_DESCRIPTION','SECONDHEAT_DESCRIPTION','HOTWATER_DESCRIPTION',
                       'LIGHTING_DESCRIPTION', 
                       #'AIR_TIGHTNESS_DESCRIPTION' 
                      ]

for col in descriptive_columns:
    epc_49[col] = epc_49[col].str.replace(r"\|(.*)","")
    epc_49[col] = epc_49[col].str.replace(r"\|(.*)","")
    epc_49[col] = epc_49[col].str.replace('  ',' ')
    epc_49[col] = epc_49[col].str.lower()
    epc_49[col] = epc_49[col].str.replace(", as built","")
    epc_49[col] = epc_49[col].str.replace("assumed","")
    epc_49[col] = epc_49[col].str.replace('(','') 
    epc_49[col] = epc_49[col].str.replace(')','')
    epc_49[col] = epc_49[col].str.replace("[ \t]+$","")
    epc_49[col] = epc_49[col].str.replace("w/m?k","w/m²k")
    
    epc_49[col] = epc_49[col].str.replace('Average thermal transmittance 1 ','Average thermal transmittance 1.00 ')
    epc_49[col] = epc_49[col].str.replace('Average thermal transmittance =','Average thermal transmittance')    # standardising the unit used
    epc_49[col] = epc_49[col].str.replace(r"w(.*?)k",'w/m²k')

In [12]:
def roof_floor_cleanup(df):
    '''cleaning ROOF_DESCRIPTION and FLOOR_DESCRIPTION'''

   # standardising descriptions
    df['ROOF_DESCRIPTION'] = df['ROOF_DESCRIPTION'].str.replace("another dwelling above","other premises above")
    df['ROOF_DESCRIPTION'] = df['ROOF_DESCRIPTION'].str.replace("+","")
    
    df['FLOOR_DESCRIPTION'] = df['FLOOR_DESCRIPTION'].str.replace("another dwelling above","other premises above")
    df['FLOOR_DESCRIPTION'] = df['FLOOR_DESCRIPTION'].str.replace("another dwelling below","other premises below")
    df['FLOOR_DESCRIPTION'] = df['FLOOR_DESCRIPTION'].str.replace("  \+"," +")
    df['FLOOR_DESCRIPTION'] = df['FLOOR_DESCRIPTION'].str.replace("[ \t]+$","")
        
    return df 
epc_49 = roof_floor_cleanup(epc_49)

In [13]:
def insulation_extract(df):
    '''takes the second part after ',' that indicates insulation in description columns '''
    
    df['wall_insulation'] = df['WALL_DESCRIPTION'].str.extract('(?<=,\s)([^,]+)')
    df['roof_insulation'] = df['ROOF_DESCRIPTION'].str.extract('(?<=,\s)([^,]+)')
    df['floor_insulation'] = df['FLOOR_DESCRIPTION'].str.extract('(?<=,\s)([^,]+)')
    df[['wall_insulation','roof_insulation','floor_insulation']] = df[['wall_insulation','roof_insulation','floor_insulation']].apply(lambda x: x.fillna(x.value_counts().index[0]))

    return df
epc_49 = insulation_extract(epc_49)

In [14]:
def wall_insulation(df):
    '''returns df with a new column 'insulated_wall' classifying wall insulation 1 if insulated 0 if not'''
     ##=============insulated=============##
    df.loc[df['wall_insulation'].str.contains('with external insulation') , 'insulated_wall'] = 1
    df.loc[df['wall_insulation'].str.contains('with internal insulation') , 'insulated_wall'] = 1
    df.loc[df['wall_insulation'].str.contains('with additional insulation') , 'insulated_wall'] = 1
    df.loc[df['wall_insulation'].str.contains('partial insulation') , 'insulated_wall'] = 1
    df.loc[df['wall_insulation'].str.contains('with insulation') , 'insulated_wall'] = 1
    df.loc[df['wall_insulation'].str.contains('filled cavity') , 'insulated_wall'] = 1
    df.loc[df['wall_insulation'].str.contains('insulated') , 'insulated_wall'] = 1
    df.loc[df['wall_insulation'].str.contains('average thermal transmittance') , 'insulated_wall'] = 1

     ##=============no insulation=============##
    df.loc[df['wall_insulation'].str.contains('no insulation') , 'insulated_wall'] = 0

    return df
epc_49 = wall_insulation(epc_49)

In [15]:
epc_49['wall_insulation'].value_counts()

no insulation                            23092
insulated                                14517
filled cavity                             9506
with external insulation                  3311
partial insulation                        2913
with internal insulation                   853
with additional insulation                 144
filled cavity and external insulation      104
filled cavity and internal insulation       55
Name: wall_insulation, dtype: int64

In [16]:
def roof_insulation(df):
    '''returns df with a new column 'insulated_roof' classifying wall insulation 1 if insulated 0 if not'''    
    ##=============insulated=============##
    df.loc[df['roof_insulation'].str.contains('loft insulation') , 'insulated_roof'] = 1
    df.loc[df['roof_insulation'].str.contains('insulated') , 'insulated_roof'] = 1
    df.loc[df['roof_insulation'].str.contains('limited insulation') , 'insulated_roof'] = 1
    df.loc[df['roof_insulation'].str.contains('with additional insulation') , 'insulated_roof'] = 1
    df.loc[df['roof_insulation'].str.contains('average thermal') , 'insulated_roof'] = 1

    ##=============no insulation=============##
    df.loc[df['roof_insulation'].str.contains('no insulation') , 'insulated_roof'] = 0

    return df
epc_49 = roof_insulation(epc_49)

In [17]:
epc_49['roof_insulation'].value_counts()

300 mm loft insulation        26272
200 mm loft insulation         5259
250 mm loft insulation         4165
no insulation                  3353
150 mm loft insulation         3209
270 mm loft insulation         2946
100 mm loft insulation         2639
insulated                      2594
limited insulation             1524
350 mm loft insulation          654
400 mm loft insulation          430
50 mm loft insulation           418
ceiling insulated               399
75 mm loft insulation           370
insulated at rafters            154
25 mm loft insulation            80
12 mm loft insulation            27
0 mm loft insulation              1
with additional insulation        1
Name: roof_insulation, dtype: int64

In [18]:
def floor_insulation(df):
    '''returns df with a new column 'insulated_floor' classifying wall insulation 1 if insulated 0 if not'''    
    ##=============insulated=============##
    df.loc[df['floor_insulation'].str.contains('insulated') , 'insulated_floor'] = 1
    df.loc[df['floor_insulation'].str.contains('limited insulation') , 'insulated_floor'] = 1
    df.loc[df['floor_insulation'].str.contains('average thermal') , 'insulated_floor'] = 1
    ##=============no insulation=============##
    df.loc[df['floor_insulation'].str.contains('no insulation') , 'insulated_floor'] = 0
    
    return df
epc_49 = floor_insulation(epc_49)

In [20]:
epc_49['floor_insulation'].value_counts()

no insulation         43702
insulated              8770
limited insulation     2023
Name: floor_insulation, dtype: int64

In [66]:
def wall_type(df):
    '''returns df with a new column 'wall_type' indicating type/ material of the walls''' 
    
    df['wall_type'] = df['WALL_DESCRIPTION'].str.extract('^([^,]+)')
    df.loc[df['wall_type'].str.contains('average ') , 'wall_type'] = 'Unknown'
    df.loc[df['wall_type'].str.contains('park home wall') , 'wall_type'] = 'park home wall'
    return df
epc_49 = wall_type(epc_49)

In [67]:
def roof_type(df):
    '''returns df with a new column 'roof_type' indicating type/ material of the roofs'''    
        
    df.loc[df['ROOF_DESCRIPTION'].str.contains('pitched') , 'roof_type'] = 'Pitched'
    df.loc[df['ROOF_DESCRIPTION'].str.contains('flat') , 'roof_type'] = 'Flat'
    df.loc[df['ROOF_DESCRIPTION'].str.contains('roof room') , 'roof_type'] = 'Roof rooms'
    df.loc[df['ROOF_DESCRIPTION'].str.contains('thatched') , 'roof_type'] = 'Thatched'
    df.loc[df['ROOF_DESCRIPTION'].str.contains('average thermal transmittance') , 'roof_type'] = 'Unknown'
    df.loc[df['ROOF_DESCRIPTION'].str.contains('other premises above') , 'roof_type'] = 'Unknown'

    return df
epc_49 = roof_type(epc_49)

In [68]:
def floor_type(df):
    '''returns df with a new column 'floor_type' indicating type/ material of the floors'''    

    df.loc[df['FLOOR_DESCRIPTION'].str.contains('suspended') , 'floor_type'] = 'Suspended'
    df.loc[df['FLOOR_DESCRIPTION'].str.contains('solid') , 'floor_type'] = 'Solid'
    df.loc[df['FLOOR_DESCRIPTION'].str.contains('to unheated space') , 'floor_type'] = 'To unheated space'
    df.loc[df['FLOOR_DESCRIPTION'].str.contains('to external air') , 'floor_type'] = 'To external air'
    df.loc[df['FLOOR_DESCRIPTION'].str.contains('average thermal transmittance') , 'floor_type'] = 'Unknown'
    df.loc[df['FLOOR_DESCRIPTION'].str.contains('other premises below') , 'floor_type'] = 'Unknown'
    df.loc[df['FLOOR_DESCRIPTION'].str.contains('other premises above') , 'floor_type'] = 'Unknown'
    
    return df
epc_49 = floor_type(epc_49)

In [21]:
def windows_glazing(df):
    '''returns a df with new column 'windows_glazing' indicating the type of windows glazing ''' 
    df['windows_glazing'] = df['WINDOWS_DESCRIPTION'].str.replace("description:","")
    df['windows_glazing'] = df['windows_glazing'].str.replace("fully","")
    df['windows_glazing'] = df['windows_glazing'].str.replace("partial","")
    df['windows_glazing'] = df['windows_glazing'].str.replace("mostly","")
    df['windows_glazing'] = df['windows_glazing'].str.replace("full","")
    df['windows_glazing'] = df['windows_glazing'].str.replace("some","")
    df['windows_glazing'] = df['windows_glazing'].str.replace("^\s+","")
    df['windows_glazing'] = df['windows_glazing'].str.replace("triple glazing","triple glazed")
    df['windows_glazing'] = df['windows_glazing'].str.replace("multiple glazing throughout","multiple glazing")
    df['windows_glazing'] = df['windows_glazing'].str.replace("double glazing","double glazed")
        
    return df
epc_49 = windows_glazing(epc_49)

In [23]:
epc_49['windows_glazing'].value_counts()

double glazed               45703
high performance glazing     6287
single glazed                1679
triple glazed                 572
secondary glazing             221
multiple glazing               33
Name: windows_glazing, dtype: int64

In [24]:
def mainheat_sys(df):
    df['main_heat_system'] = df['MAINHEAT_DESCRIPTION'].str.extract('^([^,]+)')
#     df['main_heat_system'] = df['main_heat_system'].str.replace("no system present: ","")
    df['main_heat_system'] = df['main_heat_system'].str.replace("^[ \t]+","")
    df['main_heat_system'] = df['main_heat_system'].fillna(df['main_heat_system'].value_counts().index[0])

    df['MHS_boiler'] = df['main_heat_system'].str.contains('boiler')
    df['MHS_radiators'] = df['main_heat_system'].str.contains('radiators')
    df['MHS_electric heaters'] = df['main_heat_system'].str.contains('electric heaters')
    df['MHS_electric heaters'] = df['main_heat_system'].str.contains('electric underfloor heating')
    df['MHS_electric heaters'] = df['main_heat_system'].str.contains('electric ceiling heating')
    df['MHS_electric heaters'] = df['main_heat_system'].str.contains('electric storage heaters')
    df['MHS_electric heaters'] = df['main_heat_system'].str.contains('portable electric heaters  for most rooms')
    
    df['MHS_heat pump'] = df['main_heat_system'].str.contains('ground source heat pump')
    df['MHS_heat pump'] = df['main_heat_system'].str.contains('air source heat pump')
    df['MHS_heat pump'] = df['main_heat_system'].str.contains('water source heat pump')

    df['MHS_room heaters'] = df['main_heat_system'].str.contains('room heaters')
    df['MHS_community scheme'] = df['main_heat_system'].str.contains('community scheme')
    df['MHS_underfloor heating'] = df['main_heat_system'].str.contains('underfloor heating')
    df['MHS_warm air'] = df['main_heat_system'].str.contains('warm air')
    
    #====================#
    df['main_heat_system_2'] = df['MAINHEAT_DESCRIPTION'].str.extract('(?<=,\s)([^,]+)')
    df['main_heat_system_2'] = df['main_heat_system_2'].fillna(df['main_heat_system_2'].value_counts().index[0])
    
    df['MHS_mains gas'] = df['main_heat_system_2'].str.contains('mains gas')
    df['MHS_None'] = df['main_heat_system_2'].str.contains('None')
    df['MHS_electric'] = df['main_heat_system_2'].str.contains('electric')
    df['MHS_oil'] = df['main_heat_system_2'].str.contains('oil ')
    df['MHS_radiators'] = df['main_heat_system_2'].str.contains('radiators')
    df['MHS_underfloor heating '] = df['main_heat_system_2'].str.contains('underfloor')
    df['MHS_underfloor heating '] = df['main_heat_system_2'].str.contains('underfloor heating')
    df['MHS_lpg'] = df['main_heat_system_2'].str.contains('lpg')
    df['MHS_lpg'] = df['main_heat_system_2'].str.contains('bottled lpg')
    df['MHS_dual fuel mineral and wood '] = df['main_heat_system_2'].str.contains('dual fuel mineral and wood ')
    df['MHS_warm air'] = df['main_heat_system_2'].str.contains('warm air')
    df['MHS_wood pellets_logs'] = df['main_heat_system_2'].str.contains('wood pellets')
    df['MHS_wood pellets_logs'] = df['main_heat_system_2'].str.contains('wood logs')
    df['other'] = df['main_heat_system_2'].str.contains('wood chips')
    df['MHS_coal'] = df['main_heat_system_2'].str.contains('coal')
    df['MHS_other'] = df['main_heat_system_2'].str.contains('electricaire')
    df['MHS_other'] = df['main_heat_system_2'].str.contains('anthracite')
    df['MHS_other'] = df['main_heat_system_2'].str.contains('smokeless fuel')
    
    df['MHS_other'] = df['main_heat_system_2'].str.contains('fan coil units')
    df['MHS_other'] = df['main_heat_system_2'].str.contains('b30k')
    df['MHS_other'] = df['main_heat_system_2'].str.contains('lng')      
    

    return df
epc_49 = mainheat_sys(epc_49)

In [38]:
epc_49['main_heat_system_2'].value_counts()

mains gas                           45646
electric                             2869
oil                                  2105
radiators                            1809
underfloor                            729
lpg                                   518
systems with radiators                209
dual fuel mineral and wood            180
warm air                              102
wood pellets                           87
coal                                   68
wood logs                              53
underfloor heating                     33
underfloor heating and radiators       26
electricaire                           22
anthracite                             21
smokeless fuel                          6
wood chips                              4
fan coil units                          4
b30k                                    2
bottled lpg                             1
lng                                     1
Name: main_heat_system_2, dtype: int64

In [71]:
epc_49= epc_49.drop(['main_heat_system', 'main_heat_system_2', 'wall_insulation','roof_insulation','floor_insulation'],axis=1 )

In [72]:
def main_heat_control(df):

    df['MAINHEATCONT_DESCRIPTION'] = df['MAINHEATCONT_DESCRIPTION'].str.replace("2207","")
    df['MAINHEATCONT_DESCRIPTION'] = df['MAINHEATCONT_DESCRIPTION'].str.replace("room thermostats","room thermostat")
    df['MAINHEATCONT_DESCRIPTION'] = df['MAINHEATCONT_DESCRIPTION'].str.replace("programmer and at least two room stats","room thermostat")
    df['MAINHEATCONT_DESCRIPTION'] = df['MAINHEATCONT_DESCRIPTION'].str.replace("room thermostat","room thermostat(s)") 
    df['mainheat_cont_system'] = df['MAINHEATCONT_DESCRIPTION'].str.replace("^\s+","")
    
#     df['mainheat_cont_system'] = df['MAINHEATCONT_DESCRIPTION'].str.extract('^([^,]+)')
#     df['mainheat_cont_system'] = df['mainheat_cont_system'].str.replace("^[ \t]+","")
    df['mainheat_cont_system'] = df['MAINHEATCONT_DESCRIPTION'].fillna(df['mainheat_cont_system'].value_counts().index[0])

    df['MHCS_programmer'] = df['mainheat_cont_system'].str.contains('programmer')
    df['MHCS_time and temperature zone control'] = df['mainheat_cont_system'].str.contains('time and temperature zone control')
    df['MHCS_manual charge control'] = df['mainheat_cont_system'].str.contains('manual charge control')
    df['MHCS_room thermostat(s)'] = df['mainheat_cont_system'].str.contains('room thermostat(s)')
    df['MHCS_room thermostat(s)'] = df['mainheat_cont_system'].str.contains('at least two room thermostat(s)')
    df['MHCS_appliance thermostats'] = df['mainheat_cont_system'].str.contains('appliance thermostats')
    df['MHCS_controls for high heat retention storage heaters'] = df['mainheat_cont_system'].str.contains('controls for high heat retention storage heaters')
    df['MHCS_automatic charge control'] = df['mainheat_cont_system'].str.contains('automatic charge control')
    df['MHCS_charging system linked to use of community heating'] = df['mainheat_cont_system'].str.contains('charging system linked to use of community heating')
    df['MHCS_flat rate charging'] = df['mainheat_cont_system'].str.contains('flat rate charging')
    df['MHCS_trvs'] = df['mainheat_cont_system'].str.contains('trvs')
    df['MHCS_bypass'] = df['mainheat_cont_system'].str.contains('bypass')
    df['MHCS_none'] = df['mainheat_cont_system'].str.contains('none')
    df['MHCS_no time or thermostatic control of room temperature'] = df['mainheat_cont_system'].str.contains('no time or thermostatic control of room temperature')
    df['MHCS_other'] = df['mainheat_cont_system'].str.contains('temperature zone control')
    df['MHCS_other'] = df['mainheat_cont_system'].str.contains('celect control')
    df['MHCS_boiler energy manager'] = df['mainheat_cont_system'].str.contains('boiler energy manager')
    df['MHCS_no time or thermostatic control of room temperature'] = df['mainheat_cont_system'].str.contains('no thermostatic control of room temperature')

    return df
epc_49 = main_heat_control(epc_49)

In [73]:
epc_49= epc_49.drop(['mainheat_cont_system'],axis=1 )

In [74]:
def hotwater_sys(df):
    df['hotwater_system'] = df['HOTWATER_DESCRIPTION']
    df['hotwater_system'] = df['hotwater_system'].fillna(df['hotwater_system'].value_counts().index[0])
    
    df['hws_from main system'] = df['hotwater_system'].str.contains('from main system')
    df['hws_from secondary system'] = df['hotwater_system'].str.contains('from secondary system')
    df['hws_community scheme'] = df['hotwater_system'].str.contains('community scheme')
    df['hws_electric immersion'] = df['hotwater_system'].str.contains('electric immersion')
    df['hws_electric instantaneous at point of use'] = df['hotwater_system'].str.contains('electric instantaneous at point of use')
    df['hws_heat pump'] = df['hotwater_system'].str.contains('electric heat pump for water heating only')
    df['hws_heat pump'] = df['hotwater_system'].str.contains('electric heat pump')
    df['hws_gas multipoint'] = df['hotwater_system'].str.contains('gas multipoint')
    df['hws_gas instantaneous at point of use '] = df['hotwater_system'].str.contains('gas instantaneous at point of use')
    df['hws_other'] = df['hotwater_system'].str.contains('solid fuel boiler/circulator')
    df['hws_other'] = df['hotwater_system'].str.contains('oil boiler/circulator')
    df['hws_other'] = df['hotwater_system'].str.contains('gas boiler/circulator')
    df['hws_other'] = df['hotwater_system'].str.contains('oil range cooker')
    df['hws_other'] = df['hotwater_system'].str.contains('solid fuel range cooker')
    df['hws_other'] = df['hotwater_system'].str.contains('gas range cooker')
    #2nd part
    df['hws_None'] = df['hotwater_system'].str.contains('None')
    df['hws_off-peak'] = df['hotwater_system'].str.contains('off-peak')
    df['hws_no cylinder thermostat'] = df['hotwater_system'].str.contains('no cylinder thermostat')
    df['hws_standard tariff'] = df['hotwater_system'].str.contains('standard tariff')
    df['hws_flue gas heat recovery'] = df['hotwater_system'].str.contains('flue gas heat recovery')
    df['hws_plus solar'] = df['hotwater_system'].str.contains('plus solar')
    df['hws_waste water heat recovery'] = df['hotwater_system'].str.contains('waste water heat recovery')

    return df    
epc_49 = hotwater_sys(epc_49)

In [75]:
epc_49= epc_49.drop(['hotwater_system'],axis=1 )

In [76]:
def lighting_perc(df):   
    df['low_energy_lighting_perc'] = df['LIGHTING_DESCRIPTION'].str.findall(r'(\d*)\%')
    df['low_energy_lighting_perc'] = round(df['low_energy_lighting_perc'].str[0].astype(float),-1)
    df['low_energy_lighting_perc'] = df['low_energy_lighting_perc'].fillna(df['low_energy_lighting_perc'].value_counts().index[0])
    df['low_lighting'] = df.apply(lambda row: 'low energy lighting %d%% of fixed outlets' % (int(row['low_energy_lighting_perc'])) if '%' in str(row['LIGHTING_DESCRIPTION']) else row['LIGHTING_DESCRIPTION'],axis=1)
    df['low_lighting'] = df['low_lighting'].str.replace("low energy lighting in all fixed outlets","low energy lighting 100% of fixed outlets")
    df['low_lighting'] = df['low_lighting'].str.replace("no low energy lighting","low energy lighting 0% of fixed outlets")
    return df
epc_49 = lighting_perc(epc_49)

In [77]:
epc_49 = epc_49.drop(['low_energy_lighting_perc'], axis=1)

In [78]:
def secondheat(df):
    df['second_heat'] = df['SECONDHEAT_DESCRIPTION']
    df['second_heat'] = df['SECONDHEAT_DESCRIPTION'].fillna(df['second_heat'].value_counts().index[0])

    df['SH_room heaters'] = df['second_heat'].str.contains('room heaters')
    df['SH_none'] = df['second_heat'].str.contains('none')
    df['SH_electric'] = df['second_heat'].str.contains('electric')
    df['SH_mains gas'] = df['second_heat'].str.contains('mains gas')
    df['SH_portable electric heaters'] = df['second_heat'].str.contains('portable electric heaters')
    df['SH_dual fuel mineral and wood'] = df['second_heat'].str.contains('dual fuel mineral and wood')
    df['SH_wood logs'] = df['second_heat'].str.contains('wood logs')
    df['SH_coal'] = df['second_heat'].str.contains('coal')
    df['SH_lpg'] = df['second_heat'].str.contains('lpg')
    df['SH_wood pellets'] = df['second_heat'].str.contains('wood pellets')
    df['SH_smokeless fuel'] = df['second_heat'].str.contains('smokeless fuel')
    df['SH_anthracite'] = df['second_heat'].str.contains('anthracite')
    df['SH_bioethanol'] = df['second_heat'].str.contains('bioethanol')
    df['SH_oil'] = df['second_heat'].str.contains('oil')

    return df
epc_49 = secondheat(epc_49)

In [79]:
epc_49= epc_49.drop(['second_heat'],axis=1 )

In [80]:
#combine cate variables 
built_dict = dict.fromkeys(['Mid-Terrace','End-Terrace','Enclosed Mid-Terrace','Enclosed End-Terrace'],'terraced')
built_dict1 = dict.fromkeys(['Semi-Detached','Detached'],'detached')
built_dict.update(built_dict1)

In [81]:
epc_49['built_form'] =epc_49['Built Form'].replace(built_dict)

In [82]:
epc_49= epc_49.drop(['Built Form'], axis=1)

In [83]:
#dropping description attributes
epc_49 = epc_49.drop(['WALL_DESCRIPTION', 'ROOF_DESCRIPTION', 'FLOOR_DESCRIPTION',
                      'WINDOWS_DESCRIPTION', 'MAINHEAT_DESCRIPTION','MAINHEATCONT_DESCRIPTION',
                      'SECONDHEAT_DESCRIPTION', 'HOTWATER_DESCRIPTION', 'LIGHTING_DESCRIPTION'] , axis=1)

In [84]:
epc_49.shape

(54495, 93)

In [85]:
epc_49.dtypes

Total floor area (m²)                                       float64
Total current energy costs over 3 years (£)                 float64
Current energy efficiency rating band                        object
Current heating costs over 3 years (£)                      float64
Current hot water costs over 3 years (£)                    float64
Current lighting costs over 3 years (£)                     float64
Part 1 Construction Age Band                                 object
Part 1 Floor 0 Room Height                                  float64
Fixed Lighting Outlets Count                                float64
Low Energy Lighting Outlets Count                           float64
Low Energy Lighting %                                       float64
Glazed Area                                                 float64
Habitable Room Count                                        float64
Heated Room Count                                           float64
Mechanical Ventilation                          

In [86]:
epc_49['insulated_wall'] = epc_49['insulated_wall'].astype(int)
epc_49['insulated_floor'] = epc_49['insulated_floor'].astype(int)
epc_49['insulated_roof'] = epc_49['insulated_roof'].astype(int)

In [87]:
epc_49.head()

Total floor area (m²)  Total current energy costs over 3 years (£)  \
0                   45.0                                       3771.0   
1                  143.0                                       2793.0   
2                   65.0                                       1947.0   
3                   49.0                                       1158.0   
4                  212.0                                       6477.0   

  Current energy efficiency rating band  \
0                                     E   
1                                     C   
2                                     C   
3                                     C   
4                                     E   

   Current heating costs over 3 years (£)  \
0                                  2478.0   
1                                  2112.0   
2                                  1332.0   
3                                   816.0   
4                                  5157.0   

   Current hot water costs over 3 years (£)  \
0                                    1176.0   
1                                     375.0   
2                                     441.0   
3                                     195.0   
4                                     819.0   

   Current lighting costs over 3 years (£) Part 1 Construction Age Band  \
0                                    117.0                  before 1919   
1                                    306.0                    2003-2007   
2                                    174.0                  before 1919   
3                                    147.0                    1999-2002   
4                                    501.0                  before 1919   

   Part 1 Floor 0 Room Height  Fixed Lighting Outlets Count  \
0                        3.10                           6.0   
1                        2.40                          16.0   
2                        2.45                           6.0   
3                        2.40                          11.0   
4                        2.80                          14.0   

   Low Energy Lighting Outlets Count  Low Energy Lighting %  Glazed Area  \
0                                6.0                  100.0          1.0   
1                               16.0                  100.0          1.0   
2                                6.0                  100.0          1.0   
3                               11.0                  100.0          1.0   
4                                9.0                   64.0          1.0   

   Habitable Room Count  Heated Room Count Mechanical Ventilation Meter Type  \
0                   2.0                2.0                natural       dual   
1                   7.0                7.0                natural     Single   
2                   3.0                3.0                natural     Single   
3                   2.0                2.0                natural     Single   
4                   7.0                7.0                natural     Single   

  Solar Water Heating            Tenure   Transaction Type Property Type  \
0                   N    owner-occupied  none of the above          Flat   
1                   N    owner-occupied      marketed sale         House   
2                   N    owner-occupied      marketed sale          Flat   
3                   N  rented (private)  none of the above          Flat   
4                   N    owner-occupied      marketed sale         House   

   insulated_wall  insulated_roof  insulated_floor               wall_type  \
0               0               1                0  sandstone or limestone   
1               1               1                1             cavity wall   
2               0               1                0  sandstone or limestone   
3               1               1                1             cavity wall   
4               1               1                1    granite or whinstone   

  roof_type floor_type windows_glazing  MHS_boiler  MHS_r

In [88]:
objList = epc_49.select_dtypes(include = "object").columns
# Label Encoding for object to numeric conversion
epc_encoded = epc_49.copy()
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for feat in objList:
    epc_encoded[feat] = le.fit_transform(epc_49[feat].astype(str))
# epc_encoded.info()

In [89]:
epc_49.to_csv('epc_encoded.csv')  